In [ ]:
import sys

sys.path.append("../")

In [ ]:
import re

import data.drawings.make_tasks as drawing_tasks

In [ ]:
BASH_FILE = "../run_drawings_experiments.sh"

DRAWINGS_DOMAINS = [f"drawings_{domain}" for domain in drawing_tasks.TASK_DOMAINS]
print(DRAWINGS_DOMAINS)

EXPERIMENT_TYPES = [
    "stitch",
    "oracle",
    "oracle_train_test",
    "stitch_codex",
    "stitch_codex_language",
]
print(EXPERIMENT_TYPES)

In [ ]:
RANDOM_SEEDS = ["111", "222", "333"]

cmds = []
for experiment_type in EXPERIMENT_TYPES:
    for domain in DRAWINGS_DOMAINS:
        cmd = (
            f"python run_iterative_experiment.py --domain {domain} --experiment_type {experiment_type} "
            + "--stitch_params '{\"iterations\": 10}' "
        )
        if "codex" in experiment_type:
            seeds = RANDOM_SEEDS

            codex_params = {
                "n_tasks_per_prompt": 25,
                "n_samples": 50,
                "n_samples_per_query": 5,
                "final_task_origin": "default",
                "body_task_types": ["programs"],
                "final_task_types": ["programs"],
            }

            if "language" in experiment_type:
                codex_params.update(
                    {
                        "body_task_types": ["language", "programs"],
                        "final_task_types": ["language"],
                    }
                )
            cmd += f"--codex_params {repr(codex_params)} "
            cmd += "--use_cached "
        else:
            seeds = [RANDOM_SEEDS[0]]

        cmd += f"--random_seeds {' '.join(seeds)} "
        cmd += "\n"
        cmds.append(cmd)

print(cmds[0])

In [ ]:
with open(BASH_FILE, "w") as f:
    f.writelines(cmds)